In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
!pwd
os.chdir('gdrive/My Drive/DQN/')
!pwd
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content
/content/gdrive/My Drive/DQN
configs  doc  dqn.ipynb  drl_atari  LICENSE  play.py  README.md  train.py


In [2]:
config_file = "/configs/long_dqn.yml"
config_file

'/configs/long_dqn.yml'

In [3]:
#!/usr/bin/env python3
import argparse
import logging
import yaml
import gym
import os
import shutil
import tensorflow as tf
import numpy as np

from drl_atari.models import DQN
from drl_atari import utils
from drl_atari.multiple_sync_env import MultipleSyncEnv
from drl_atari.experience_replay import ExperienceReplay

logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

# def parse_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('config_file')
#     # parser.add_argument('basic_dqn.yml')
#     return parser.parse_args()

# def prepare_output_dir(cfg, config_file):
#     output_dir = utils.output_dir(cfg)
#     if os.path.exists(output_dir):
#         raise ValueError("Output directory '%s' already exits!" % output_dir)
#     os.makedirs(output_dir)
#     shutil.copy2(config_file, output_dir)
#     utils.save_code_info(output_dir)

# def main(args):

def main():

    # config_file = args.config_file
    config_file = 'configs/long_dqn.yml'

    log.info("Using '" + str(config_file) + "' as a config file.")
    with open(config_file, 'r') as yml_file:
        cfg = yaml.load(yml_file)
    # prepare_output_dir(cfg, config_file)

    env = gym.make(cfg['env_id'])
    model_input_state_shape, model_output_action_shape = utils.in_out_shapes(cfg, env)

    if cfg['model_class'] == 'DQN':

        num_state_imgs = model_input_state_shape[-1]
        update_frequency = cfg['update_frequency']
        replay_start_size = cfg['replay_start_size']
        batch_size = cfg['batch_size']

        dqn = DQN(cfg, input_shape=model_input_state_shape, output_shape=model_output_action_shape)
        dqn.build()

        experience_replay = ExperienceReplay(max_size=cfg['max_replay_memory_size'])

        with tf.Session() as sess:

            dqn.set_session(sess)

            sess.run(tf.global_variables_initializer())
            sess.run(tf.local_variables_initializer())

            dqn.update_target_network()

            done = True
            episode = 0
            total_reward = 0.0
            total_actions = 0

            while True:

                if done:
                    if episode > 0:
                        log.info('Episode: ' + str(episode))
                        log.info('Total reward: ' + str(total_reward))
                        dqn.report_score(total_reward)

                    episode += 1
                    total_reward = 0.0
                    ob = env.reset()
                    observations = [ob]
                    for _ in range(num_state_imgs - 1):
                        ob, r, done, _ = env.step(env.action_space.sample())
                        observations.append(ob)
                    state = utils.convert_observation_list(model_input_state_shape, observations)

                exploration_prob = dqn.get_exploration_prob(total_actions)
                explore_randomly = np.random.rand() < exploration_prob or experience_replay.size() < replay_start_size
                if explore_randomly:
                    action = env.action_space.sample()
                else:
                    action = dqn.predict_act(state)[0]
                ob, r, done, _ = env.step(action)

                total_reward += r
                r = np.clip(r, -1., 1.)

                conv_ob = utils.convert_observation(model_input_state_shape, ob)
                next_state = np.concatenate([state, conv_ob], axis=-1)[:, :, :, -num_state_imgs:]

                experience_replay.add(state, action, r, done, next_state)

                if total_actions % update_frequency == 0 and experience_replay.size() >= replay_start_size:
                    rand_experiences = experience_replay.sample(batch_size)
                    states, actions, rewards, dones, next_states = zip(*rand_experiences)
                    states = np.array(states).reshape((batch_size,) + model_input_state_shape)
                    actions = np.array(actions).reshape((batch_size,))
                    rewards = np.array(rewards).reshape((batch_size,))
                    dones = np.array(dones).reshape((batch_size,))
                    next_states = np.array(next_states).reshape((batch_size,) + model_input_state_shape)

                    dqn.update_params(states, actions, rewards, dones, next_states)

                total_actions += 1
                state = next_state
    else:
        raise ValueError("model_class: %s not supported" % cfg['model_class'])


if __name__ == "__main__":
#    main(parse_args())
    main()

INFO:__main__:Using 'configs/long_dqn.yml' as a config file.


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Please use `layer.__call__` method instead.


...

INFO:tensorflow:Summary name main/state_actions_values/bias:0-grad-clip is illegal; using main/state_actions_values/bias_0-grad-clip instead.


INFO:tensorflow:Summary name main/state_actions_values/bias:0-grad-clip is illegal; using main/state_actions_values/bias_0-grad-clip instead.



INFO:drl_atari.models.dqn:Target network updated!
INFO:__main__:Episode: 1
INFO:__main__:Total reward: -21.0



INFO:__main__:Episode: 2
INFO:__main__:Total reward: -21.0
INFO:__main__:Episode: 3
INFO:__main__:Total reward: -20.0
INFO:__main__:Episode: 4
INFO:__main__:Total reward: -21.0
INFO:__main__:Episode: 5
INFO:__main__:Total reward: -21.0
INFO:__main__:Episode: 6
INFO:__main__:Total reward: -20.0
INFO:__main__:Episode: 7
INFO:__main__:Total reward: -19.0
INFO:__main__:Episode: 8
INFO:__main__:Total reward: -19.0
INFO:__main__:Episode: 9
INFO:__main__:Total reward: -21.0
INFO:__main__:Episode: 10
INFO:__main__:Total reward: -21.0
INFO:__main__:Episode: 11
INFO:__main__:Total reward: -20.0
INFO:__main__:Episode: 12
INFO:__main__:Total reward: -20.0
INFO:__main__:Episode: 13
INFO:__main__:Total reward: -20.0
INFO:__main__:Episode: 14
INFO:__main__:Total reward: -20.0
INFO:__main__:Episode: 15
INFO:__main__:Total reward: -20.0
INFO:__main__:Episode: 16
INFO:__main__:Total reward: -20.0
INFO:__main__:Episode: 17
INFO:__main__:Total reward: -21.0
INFO:__main__:Episode: 18
INFO:__main__:Total r

KeyboardInterrupt: ignored